In [2]:
import os

import numpy as np
import pandas as pd

from hyperopt import fmin, hp, tpe, STATUS_OK
from tensorflow import keras

In [5]:
train_raw = pd.read_csv('data/train.csv').drop('id', axis=1).to_numpy()
test_raw = pd.read_csv('data/test.csv', dtype=float)
split = int(train_raw.shape[0] * .6)
X_train = train_raw[:split, :14]
y_train = train_raw[:split, 14:]
X_valid = train_raw[split:, :14]
y_valid = train_raw[split:, 14:]
X_valid, X_test = np.array_split(X_valid, 2)
y_valid, y_test = np.array_split(y_valid, 2)

In [6]:
# sample size, time steps, input dimension
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
X_valid = X_valid.reshape(X_valid.shape[0], X_valid.shape[1], 1)
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)

In [36]:
# baseline_model = keras.models.Sequential([
#     keras.layers.Conv1D(128, kernel_size=7, activation='relu', input_shape=(X_train.shape[1], X_train.shape[2])),
#     keras.layers.Conv1D(256, kernel_size=3, activation='relu'),
#     keras.layers.MaxPool1D(pool_size=2),
#     keras.layers.Flatten(),
#     keras.layers.Dense(48, activation='elu'),
#     keras.layers.Dense(1, activation='relu')
# ])
baseline_model = keras.models.Sequential([
    keras.layers.Conv1D(64, kernel_size=6, activation='relu', input_shape=(X_train.shape[1], X_train.shape[2])),
    keras.layers.Dropout(.2),
    keras.layers.Conv1D(128, kernel_size=4, activation='relu'),
    keras.layers.Conv1D(256, kernel_size=2, activation='relu'),
    keras.layers.Dropout(.2),
    keras.layers.MaxPool1D(pool_size=4),
    keras.layers.Flatten(),
    keras.layers.GaussianNoise(.1),
    keras.layers.Dense(48, activation='elu'),
    keras.layers.Dense(1, activation='elu')
])

baseline_model.compile(optimizer='adam', loss='mse')
baseline_model.fit(X_train, y_train, epochs=40, batch_size=128,
                   validation_data=(X_valid, y_valid))

Epoch 1/40
1407/1407 [==============================] - 7s 4ms/step - loss: 0.9357 - val_loss: 1.1895
Epoch 2/40
1407/1407 [==============================] - 6s 4ms/step - loss: 0.5586 - val_loss: 0.9213
Epoch 3/40
1407/1407 [==============================] - 6s 4ms/step - loss: 0.5485 - val_loss: 0.6458
Epoch 4/40
1407/1407 [==============================] - 6s 4ms/step - loss: 0.5426 - val_loss: 0.7266
Epoch 5/40
1407/1407 [==============================] - 6s 4ms/step - loss: 0.5370 - val_loss: 0.5541
Epoch 6/40
1407/1407 [==============================] - 6s 4ms/step - loss: 0.5337 - val_loss: 0.5605
Epoch 7/40
1407/1407 [==============================] - 6s 4ms/step - loss: 0.5304 - val_loss: 0.5403
Epoch 8/40
1407/1407 [==============================] - 6s 4ms/step - loss: 0.5292 - val_loss: 0.5414
Epoch 9/40
1407/1407 [==============================] - 6s 4ms/step - loss: 0.5276 - val_loss: 0.5261
Epoch 10/40
1407/1407 [==============================] - 6s 4ms/step - loss: 0.525

In [23]:
X_target = test_raw.drop('id', axis=1).to_numpy()
y_target = baseline_model.predict(X_target)

baseline_model.save(os.path.join('saved_models', 'conv_baseline3'))
pd\
    .DataFrame(np.column_stack([test_raw.loc[:, 'id'], y_target]), columns=['id', 'target'])\
    .astype({'id': int})\
    .to_csv(os.path.join('submissions', 'conv_baseline3.csv'), index=False)

6250/6250 [==============================] - 8s 1ms/step


INFO:tensorflow:Assets written to: saved_models\conv_baseline3\assets


INFO:tensorflow:Assets written to: saved_models\conv_baseline3\assets
